In [6]:
import pandas as pd
from datetime import date, datetime, timedelta



ocorrencias = pd.read_csv('../data/data_2016.csv', sep = ',')

# Join all csvs in a single pandas dataFrame
ocorrencias2016 = pd.read_csv(
    'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-2016.csv', 
    sep = ',',
    on_bad_lines='skip',
    low_memory=False
)

ocorrencias2017 = pd.read_csv(
    'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-2017.csv', 
    sep = ',',
    on_bad_lines='skip',
    low_memory=False
)

ocorrencias2018 = pd.read_csv(
    'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-2018.csv', 
    sep = ',',
    on_bad_lines='skip',
    low_memory=False
)

ocorrencias2019 = pd.read_csv(
    'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-2019.csv', 
    sep = ',',
    on_bad_lines='skip',
    low_memory=False
)

ocorrencias2020 = pd.read_csv(
    'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-2020.csv', 
    sep = ',',
    on_bad_lines='skip',
    low_memory=False
)

ocorrencias = pd.concat([ocorrencias2016, ocorrencias2017, ocorrencias2018, ocorrencias2019, ocorrencias2020], axis=1)

# Replace the commas in the Latitude/Longitude columns with dots
ocorrencias['Latitude'] = pd.to_numeric(ocorrencias['Latitude'].replace(',', '.'))
ocorrencias['Longitude'] = pd.to_numeric(ocorrencias['Longitude'].replace(',', '.'))

#data_2016
#data_2016_densidade (merge com densidade_populacional_2016)
#data_2016_densidade_weekday (adition of weekday column)
#ocorrencias_clean (????????????????)
#ocorrencias_clean_extras (????????)
#ocorrencias_clean_extras_edgar (adition of holidays)



HTTPError: HTTP Error 404: Not Found

In [8]:
# Imports
import pandas as pd
import datetime 

def read_dataset(year):
    # Read the database .CSV
    df = pd.read_csv(
        f'https://raw.githubusercontent.com/centraldedados/protecao_civil/master/data/anpc-{year}.csv', 
        sep = ',', 
        on_bad_lines='skip'
    )

    # We replace the "," with "." to facilitate processing
    df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'))
    df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'))
    return df

# Joins as many datasets as possible (more than 3 causes an error due to its size)
occ = pd.concat([read_dataset(2016), read_dataset(2017), read_dataset(2018)])

# Transforms dates from a string to a DateTime object
occ['DataOcorrencia'] = pd.to_datetime(occ['DataOcorrencia'], format='%d/%m/%Y %H:%M:%S', errors='coerce') 
occ['DataFechoOperacional'] = pd.to_datetime(occ['DataFechoOperacional'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Removes the time part of the DateTime
occ['Date'] = occ['DataOcorrencia'].dt.date

# Removes rows with nulls in these columns
occ = occ.dropna(subset=["DataOcorrencia", "Latitude"])

# Drops the rows where NumeroOperacionaisTerrestresEnvolvidos is bigger than 2000
occ = occ.loc[occ['NumeroOperacionaisTerrestresEnvolvidos'] < 2000]

# Atypical values of latitude and longitude are filtered 
occ.loc[occ['Latitude'] > 43, 'Latitude'] /= 1000
occ.loc[occ['Longitude'] > 0, 'Longitude'] *= -1
occ.loc[occ['Longitude'] < -1000, 'Longitude'] /= 1000


# Atypical combinations of latitude and longitude are filtered 
occ = occ.loc[(occ['Longitude'] < -6) & (occ['Longitude'] > -10) & (occ['Latitude'] < 43) & (occ['Latitude'] > 36)]

# Removed rows older than 11 May 2016 due to inconsistency from it
occ = occ.loc[occ['Date'] >= datetime.date(year=2016, month=5, day=11)]

#data_2016
#data_2016_densidade (merge com densidade_populacional_2016)
#data_2016_densidade_weekday (adition of weekday column)
#ocorrencias_clean (????????????????)
#ocorrencias_clean_extras (????????)
#ocorrencias_clean_extras_edgar (adition of holidays)

print(occ.columns)
print(occ.head)

C:\Users\alipi\AppData\Local\Temp\ipykernel_25948\2369205207.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\alipi\AppData\Local\Temp\ipykernel_25948\2369205207.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Index(['Numero', 'DataOcorrencia', 'DataFechoOperacional', 'Natureza',
       'EstadoOcorrencia', 'Distrito', 'Concelho', 'Freguesia', 'Localidade',
       'Latitude', 'Longitude', 'NumeroMeiosTerrestresEnvolvidos',
       'NumeroOperacionaisTerrestresEnvolvidos', 'NumeroMeiosAereosEnvolvidos',
       'NumeroOperacionaisAereosEnvolvidos', 'Date'],
      dtype='object')
<bound method NDFrame.head of                Numero      DataOcorrencia DataFechoOperacional  \
698     2016030035371 2016-05-11 23:58:00  2016-05-12 00:40:00   
699     2016070010240 2016-05-11 23:43:00  2016-05-12 00:17:00   
700     2016120008794 2016-05-11 23:38:00  2016-05-12 00:04:00   
701     2016060020439 2016-05-11 23:35:00  2016-05-12 00:20:00   
702     2016150039864 2016-05-11 23:24:00  2016-05-12 00:02:00   
...               ...                 ...                  ...   
234790  2018010106781 2018-12-31 00:47:00  2018-12-31 01:40:00   
234791  2018080056438 2018-12-31 00:42:00  2018-12-31 02:23:00   
2347

In [11]:
densidade_populacional = pd.read_csv('../data/densidade_populacional_2016.csv')

# Assures uppercase compatiblity between these dataframes 
densidade_populacional["Concelho"] = densidade_populacional["Concelho"].str.upper()

# Merge them by "Concelho" matches
occ = pd.merge(occ, densidade_populacional, on = "Concelho")

# Removes extra spaces and replaces commas with dots so that Densidade is transformed into a float
occ['Densidade'] = occ['Densidade'].str.replace(' ', '').str.replace(',', '.').astype(float)

In [14]:
# create a new column "weekday" and fill it with the weekday number
occ['Weekday'] = occ['DataOcorrencia'].apply(lambda x: x.weekday())

698       2
699       2
700       2
701       2
702       2
         ..
234790    0
234791    0
234792    0
234793    0
234794    0
Name: Weekday, Length: 573254, dtype: int64


In [19]:
import csv
from datetime import date, datetime, timedelta


with open('../data/nationalHolidays.csv', 'r') as file:
    reader = csv.reader(file)
    yearlyNationalHolidays = [row[0] for row in reader]


with open('../data/regionalHolidays.csv', 'r') as file:
    reader = csv.reader(file)
    yearlyRegionalHolidays = [(row[0], row[1]) for row in reader]

# Calculates easter date for each year and all other holidays based on its date
def calculate_easter_date(year):
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    L = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * L) // 451
    month = (h + L - 7 * m + 114) // 31
    day = ((h + L - 7 * m + 114) % 31) + 1

    easter = datetime(year=year, month=month, day=day)

    carnival = easter - timedelta(days=47)
    holy_friday = easter - timedelta(days=2)
    easter_monday = easter + timedelta(days=1)
    body_of_chirst = easter + timedelta(days=60)

    return [f'{carnival.year}-0{carnival.month}-{carnival.day:02d}', 
            f'{holy_friday.year}-0{holy_friday.month}-{holy_friday.day:02d}',
            f'{easter.year}-0{easter.month}-{easter.day:02d}',
            f'{easter_monday.year}-0{easter_monday.month}-{easter_monday.day:02d}',
            f'{body_of_chirst.year}-0{body_of_chirst.month}-{body_of_chirst.day:02d}']

nationalHolidays = []
municipalHolidays = []
for year in range (2016, 2021):
    for date in yearlyNationalHolidays:
        nationalHolidays.append(f'{year}-{date}')

    for date in calculate_easter_date(year):
        nationalHolidays.append(date)

    for date in yearlyRegionalHolidays:
        municipalHolidays.append((f'{year}-{date[0]}', date[1]))

def is_holiday(row):
    date = row['DataOcorrencia'].strftime('%Y-%m-%d')
    municipality = row['Concelho'].capitalize()
    return date in nationalHolidays or (date, municipality) in municipalHolidays

occ['Holiday'] = occ.apply(lambda row: is_holiday(row), axis=1)

# Saves preparation
occ.to_csv('../data/ocorrencias_final.csv', sep = ',', index = False)



698       False
699       False
700       False
701       False
702       False
          ...  
234790    False
234791    False
234792    False
234793    False
234794    False
Name: Holiday, Length: 573254, dtype: bool


In [22]:
print(sum(occ['Holiday']))
print(len(occ))

24597
573254
